# Basic utilities

In [3]:
import random


random.seed(0)
SEED = random.randint(0, 10**6)

In [4]:
from sklearn.metrics import accuracy_score


def score(y_test, y_pred):
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Loading iris

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


data = load_iris()
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=SEED)

# Basic usage of XGBClassifier

In [6]:
from xgboost import XGBClassifier

model = XGBClassifier(objective='binary:logistic', seed=SEED, seed_per_iteration=True)


model.fit(X_train, y_train)
print(model.get_xgb_params())
y_pred = model.predict(X_test)
score(y_test, y_pred)

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'eval_metric': None,
 'gamma': None,
 'grow_policy': None,
 'hpo': False,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'seed': 885440,
 'seed_per_iteration': True}

# XGBClassifier with better hyper parameters

In [5]:
from xgboost import XGBClassifier
from math import sqrt, floor


m = len(X_train[0])
model = XGBClassifier(
    learning_rate = 0.2, # 0.0 - 1.0, log, not sure if each forest can have its own, it might lead to overfitting anyway

    max_depth = 20, # 2 - 50, 6 - 20 is another more conservative option
    subsample = 0.63, # 0.0 - 1.0
    # u can set only one of the colsample_by*
    # colsample_bytree: Optional[float] = None,
    # colsample_bylevel: Optional[float] = None,
    colsample_bynode = floor(sqrt(m))/m, # m..num_of_features, 0 - m, log?
    n_estimators = 10, # 100 - 500 = number of random forests in booster
    num_parallel_tree = 10, # 100 - 500 = number of trees in each random forest
    reg_lambda = 0, # -10. - 10.0, log = prunning of trees, higher value -> more prunning, not sure if negative values do anything
    min_child_weight = 2, # 0.0 - 10.0, log, higher value -> less options to choose from when selecting new nodes in trees

    objective = 'binary:logistic', # list at https://xgboost.readthedocs.io/en/stable/parameter.html, search for objective
    seed=SEED, 
    seed_per_iteration=True

)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score(y_test, y_pred)

Accuracy: 1.0
